<a href="https://www.kaggle.com/code/aleksandrmorozov123/nlp-with-python?scriptVersionId=155975188" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.REL
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.ALL
/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.QRY
/kaggle/input/religious-and-philosophical-texts/35895-0.txt
/kaggle/input/religious-and-philosophical-texts/pg2800.txt
/kaggle/input/religious-and-philosophical-texts/pg2680.txt
/kaggle/input/religious-and-philosophical-texts/pg10.txt
/kaggle/input/religious-and-philosophical-texts/pg17.txt


**Code to populate the documents dictionary**

In [2]:
def read_documents ():
    f = open ("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.ALL")
    merged = " "
    # the string variable merged keeps the result of merging the field identifier with its content
    
    for a_line in f.readlines ():
        if a_line.startswith ("."):
            merged += "\n" + a_line.strip ()
        else:
            merged += " " + a_line.strip ()
    # updates the merged variable using a for-loop
    
    documents = {}
    
    content = ""
    doc_id = ""
    # each entry in the dictioanry contains key = doc_id and value = content
    
    for a_line in merged.split ("\n"):
        if a_line.startswith (".I"):
            doc_id = a_line.split (" ") [1].strip()
        elif a_line.startswith (".X"):
            documents[doc_id] = content
            content = ""
            doc_id = ""
        else:
            content += a_line.strip ()[3:] + " "
    f.close ()
    return documents

# print out the size of the dictionary and the content of the very first article
documents = read_documents ()
print (len (documents))
print (documents.get ("1"))
    

1460
 18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


**Code to populate the queries dictionary**

In [3]:
def read_queries ():
    f = open ("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.QRY")
    merged = ""
    
    # merge the conten of each field with its identifier and separate different fields with lune breaks
    for a_line in f.readlines ():
        if a_line.startswith ("."):
            merged += "\n" + a_line.strip ()
        else:
            merged += " " + a_line.strip ()
    
    queries = {}
    
    # initialize queries dictionary with key = qry_id and value=content for each query in the dataset
    content = ""
    qry_id = ""
    
    for a_line in merged.split ("\n"):
        if a_line.startswith (".I"):
            if not content == "":
                queries [qry_id] = content
                content = ""
                qry_id = ""
            # add an enrty to the dictionary when you encounter an .I identifier
            qry_id = a_line.split(" ")[1].strip ()
        # otherwise, keep adding content to the content variable
        elif a_line.startswith (".W") or a_line.startswith (".T"):
            content += a_line.strip ()[3:] + " "
    queries [qry_id] = content
    f.close ()
    return queries

# print out the length of the dictionary and the content of the first query
queries = read_queries ()
print (len (queries))
print (queries.get("1"))

112
What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles? 


**Code to populate the mappings dictionary**

In [4]:
def read_mappings ():
    f = open ("/kaggle/input/cisi-a-dataset-for-information-retrieval/CISI.REL")
    mappings = {}
    
    for a_line in f.readlines ():
        voc = a_line.strip ().split ()
        key = voc[0].strip ()
        current_value = voc[1].strip()
        value = []
        # update the entry in the mappings dictionary with the current value
        if key in mappings.keys ():
            value = mappings.get (key)
        value.append (current_value)
        mappings [key] = value
    f.close ()
    return mappings

# print out some information about the mapping data structure
mappings = read_mappings ()
print (len (mappings))
print (mappings.keys ())
print (mappings.get ("1"))

76
dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '37', '39', '41', '42', '43', '44', '45', '46', '49', '50', '52', '54', '55', '56', '57', '58', '61', '62', '65', '66', '67', '69', '71', '76', '79', '81', '82', '84', '90', '92', '95', '96', '97', '98', '99', '100', '101', '102', '104', '109', '111'])
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '195', '215', '269', '291', '320', '429', '465', '466', '482', '483', '510', '524', '541', '576', '582', '589', '603', '650', '680', '711', '722', '726', '783', '813', '820', '868', '869', '894', '1162', '1164', '1195', '1196', '1281']


**Preprocess the data in documents and queries**

In [5]:
# import required libraries
import nltk
from nltk import word_tokenize

# text is converted to lowercase and split into words
def get_words (text):
    word_list = [word for word in word_tokenize (text.lower ())]
    return word_list
    
doc_words = {}
qry_words = {}

for doc_id in documents.keys ():
    doc_words [doc_id] = get_words (documents.get (doc_id))
for qry_id in queries.keys ():
    # entries in both documents and queries are represented as word lists
    qry_words [qry_id] = get_words (queries.get (qry_id))
    
# print out the length of the dictionaries and check the first document and the fisrt query
print (len (doc_words))
print (doc_words.get ("1"))
print (len (doc_words.get ("1")))
print (len (qry_words))
print (qry_words.get ("1"))
print (len (qry_words.get("1")))

1460
['18', 'editions', 'of', 'the', 'dewey', 'decimal', 'classifications', 'comaromi', ',', 'j.p.', 'the', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'dewey', 'decimal', 'classification', '.', 'the', 'first', 'edition', 'of', 'the', 'ddc', 'was', 'published', 'in', '1876', ',', 'the', 'eighteenth', 'edition', 'in', '1971', ',', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed', '.', 'in', 'spite', 'of', 'the', 'ddc', "'s", 'long', 'and', 'healthy', 'life', ',', 'however', ',', 'its', 'full', 'story', 'has', 'never', 'been', 'told', '.', 'there', 'have', 'been', 'biographies', 'of', 'dewey', 'that', 'briefly', 'describe', 'his', 'system', ',', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad', '.']
113
112
['what', 'problems', 'and', 'concerns',

**Simple Biilean search algorithm**

In [6]:
# iterate through the documents
def retrieve_documents (doc_words, query):
    docs = []
    for doc_id in doc_words.keys ():
        found = False
        i = 0
        while i<len(query) and not found: 
            word = query [i]
            if word in doc_words.get (doc_id):
                docs.append (doc_id)
                found = True
            else:
                i+=1
    return docs

# check the results
docs = retrieve_documents (doc_words, qry_words.get("3"))
print (docs [:100])
print (len (docs))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102']
1397


**Begin the preprocessing - remove stopwords and punctuation marks**

In [7]:
# import python's string module that will help remove punctuation marks
import string

# import the stopwords list
from nltk import word_tokenize
from nltk.corpus import stopwords

def process (text):
    stoplist = set (stopwords.words ('english'))
    # only add tthe words if they are not included in the stoplist and are not puctuation marks
    word_list = [word for word in word_tokenize (text.lower())
                if not word in stoplist and not word in string.punctuation]
    return word_list

# check the results of these preprocessing steps on some documents or queries
word_list = process (documents.get ("1"))
print (word_list)

['18', 'editions', 'dewey', 'decimal', 'classifications', 'comaromi', 'j.p.', 'present', 'study', 'history', 'dewey', 'decimal', 'classification', 'first', 'edition', 'ddc', 'published', '1876', 'eighteenth', 'edition', '1971', 'future', 'editions', 'continue', 'appear', 'needed', 'spite', 'ddc', "'s", 'long', 'healthy', 'life', 'however', 'full', 'story', 'never', 'told', 'biographies', 'dewey', 'briefly', 'describe', 'system', 'first', 'attempt', 'provide', 'detailed', 'history', 'work', 'spurred', 'growth', 'librarianship', 'country', 'abroad']


**Next step in preprocessing - stemming**

In [8]:
# import the stemmer
from nltk.stem.lancaster import LancasterStemmer

def process (text):
    stoplist = set (stopwords.words ('english'))
    # initialize the LancasterStemmer
    st = LancasterStemmer ()
    word_list = [st.stem(word) for word in word_tokenize (text.lower ())
                if not word in stoplist and not word in string.punctuation]
    return word_list

# check the results on some document, query, or on a list of words
word_list = process (documents.get("26"))
print (word_list)
word_list = process ("organize, organizing, organizational, organ, organic, organizer")
print (word_list)

['index', 'abstract', 'assocy', 'doyl', 'l.b', 'artic', 'discuss', 'poss', 'exploit', 'stat', 'word', 'co-occurrence', 'text', 'purpos', 'docu', 'retriev', 'co-occurrence', 'defin', 'rel', 'ment', 'process', 'auth', 'read', 'sev', 'mean', 'quantit', 'meas', 'word', 'co-occurrence', 'scrutinized', 'shown', 'strongly', 'co-occurring', 'word', 'pair', 'theref', '``', 'assocy', "''", 'stat', 'sens', 'repres', 'form', '``', 'assocy', 'map', "''", 'last', 'half', 'artic', 'pres', 'two', 'mod', 'us', 'assocy', 'map', 'lit', 'search']
['org', 'org', 'org', 'org', 'org', 'org']


**Estimate term frequency in documents and queries**

In [9]:
def get_terms (text):
    stoplist = set (stopwords.words ('english'))
    terms = {}
    st = LancasterStemmer ()
    word_list = [st.stem(word) for word in word_tokenize (text.lower ())
                if not word in stoplist and not word in string.punctuation]
    for word in word_list:
        terms [word] = terms.get (word, 0) + 1
    return terms

doc_terms = {}
qry_terms = {}
for doc_id in documents.keys ():
    doc_terms [doc_id] = get_terms (documents.get (doc_id))
for qry_id in queries.keys ():
    # populate the term frequency dictionaries for all documents and all queries
    qry_terms [qry_id] = get_terms (queries.get (qry_id))
    
# check the results
print (len (doc_terms))
print (doc_terms.get ("1"))
print (len (doc_terms.get("1")))
print (len (qry_terms))
print (qry_terms.get("1"))
print (len (qry_terms.get("1")))


1460
{'18': 1, 'edit': 4, 'dewey': 3, 'decim': 2, 'class': 2, 'comarom': 1, 'j.p.': 1, 'pres': 1, 'study': 1, 'hist': 2, 'first': 2, 'ddc': 2, 'publ': 1, '1876': 1, 'eighteen': 1, '1971': 1, 'fut': 1, 'continu': 1, 'appear': 1, 'nee': 1, 'spit': 1, "'s": 1, 'long': 1, 'healthy': 1, 'lif': 1, 'howev': 1, 'ful': 1, 'story': 1, 'nev': 1, 'told': 1, 'biograph': 1, 'brief': 1, 'describ': 1, 'system': 1, 'attempt': 1, 'provid': 1, 'detail': 1, 'work': 1, 'spur': 1, 'grow': 1, 'libr': 1, 'country': 1, 'abroad': 1}
43
112
{'problem': 1, 'concern': 1, 'mak': 1, 'describ': 1, 'titl': 3, 'difficul': 1, 'involv': 1, 'autom': 1, 'retriev': 1, 'artic': 2, 'approxim': 1, 'us': 1, 'relev': 1, 'cont': 1}
14


**Code to represent the datya in a shared space**

In [10]:
# collect the shared vocabulary of terms from documents and queries and return it as a sorted list
def collect_vocabulary ():
    all_terms = []
    for doc_id in doc_terms.keys ():
        for term in doc_terms.get (doc_id).keys():
            all_terms.append (term)
    for qry_id in qry_terms.keys ():
        for term in qry_terms.keys():
            for term in qry_terms.get(qry_id).keys():
                all_terms.append (term)
    return sorted (set (all_terms))

# print out the length of the shared vocabulary and check the first several terms in the vocabulary
all_terms = collect_vocabulary ()
print (len (all_terms))
print (all_terms [:10])

def vectorize (input_features, vocabulary):
    output = {}
    for item_id in input_features.keys ():
        features = input_features.get (item_id)
        output_vector = []
        for word in vocabulary:
            if word in features.keys ():
                output_vector.append (int (features.get (word)))
            else:
                output_vector.append (0)
        output [item_id] = output_vector
    return output

doc_vectors = vectorize (doc_terms, all_terms)
qry_vectors = vectorize (qry_terms, all_terms)

# print out some statistics on these data structures
print (len (doc_vectors))
print (len (doc_vectors.get ("1450")))
print (len (qry_vectors))
print (len (qry_vectors.get ("110")))

8881
["''", "'60", "'70", "'a", "'anyhow", "'apparent", "'b", "'basic", "'better", "'bibliograph"]
1460
8881
112
8881


**Calculate and apply inverse document frequency weighting**

In [11]:
# import library for math
import math

def calculate_idfs (vocabulary, doc_features):
    doc_idfs = {}
    for term in vocabulary:
        doc_count = 0
        for doc_id in doc_features.keys ():
            terms = doc_features.get (doc_id)
            if term in terms.keys ():
                doc_count += 1
        doc_idfs [term] = math.log (float (len (doc_features.keys ()))/
                                    float (1 + doc_count), 10)
    return doc_idfs

# check the results - we should have idf values for all terms from the vocabulary
doc_idfs = calculate_idfs (all_terms, doc_terms)
print (len (doc_idfs))
print (doc_idfs.get ("system"))

# define a function to apply idf weighing to the input_terms data structure
def vectorize_idf (input_terms, input_idfs, vocabulary):
    output = {}
    for item_id in input_terms.keys ():
        terms = input_terms.get (item_id)
        output_vector = []
        for term in vocabulary:
            if term in terms.keys ():
                # multiply the term frequencies with idf weights if the term is present in document
                output_vector.append (
                input_idfs.get (term) * float (terms.get (term)))
            else:
                output_vector.append (float (0))
        output [item_id] = output_vector
    return output

# apply idf weighing to doc_terms
doc_vectors = vectorize_idf (doc_terms, doc_idfs, all_terms)

# print out some statistics, such as the number of documents and terms
print (len (doc_vectors))
print (len (doc_vectors.get ("1460")))

8881
0.43844122348938885
1460
8881


**Run search algorithm for a given query on the set of the documents**

In [12]:
# the operator's itemgetter functionality helps sort Python dictionaries by keys or values
from operator import itemgetter

# calculate the length of the input vector
def length (vector):
    sq_length = 0
    for index in range (0, len(vector)):
        sq_length += math.pow (vector [index], 2)
    return math.sqrt (sq_length)

# calculate the dot product of two vectors
def dot_product (vector1, vector2):
    if len (vector1) == len (vector2):
        dot_prod = 0
        for index in range (0, len(vector1)):
            if not vector1 [index] == 0 and not vector2 [index] == 0:
                dot_prod += vector1 [index] * vector2 [index]
        return dot_prod
    else:
        return "Unmatching dimensionality"
    
def calculate_cosine (query, document):
    cosine = dot_product (query, document) / (length (query) * length (document))
    return cosine

query = qry_vectors.get ("3")
results = {}

for doc_id in doc_vectors.keys ():
    document = doc_vectors.get (doc_id)
    cosine = calculate_cosine (query, document)
    results [doc_id] = cosine
    
# sort the results dictionary by cosine values in descending order and return the top n results
for items in sorted (results.items (), key = itemgetter (1), reverse = True) [:44]:
    print (items [0])

469
1179
1142
1181
1190
445
1116
85
540
599
372
60
1030
241
1161
965
1191
899
137
535
456
803
95
1077
560
1095
166
544
1133
1080
640
163
837
686
1082
1297
839
1111
1428
1330
110
440
132
1137


**Estimate precision@k and ratio of cases with at least one relevant document**

In [13]:
# calculate the proportion of relevant documents from the gold standard in the top k returned results
def calculate_precision (model_output, gold_standard):
    true_pos = 0
    for item in model_output:
        if item in gold_standard:
            true_pos += 1
    return float (true_pos) / float (len (model_output))

def calculate_found (model_output, gold_standard):
    found = 0
    for item in model_output:
        if item in gold_standard:
            found = 1
    return float (found)

precision_all = 0.0
found_all = 0.0
for query_id in mappings.keys ():
    # calculate mean values across all queries
    gold_standard = mappings.get (str (query_id))
    query = qry_vectors.get (str (query_id))
    results = {}
    model_output = []
    for doc_id in doc_vectors.keys ():
        document = doc_vectors.get (doc_id)
        cosine = calculate_cosine (query, document)
        # for each document, esimate its relevance to the query with cosine similarity as before
        results [doc_id] = cosine
    # sort the results and consider only top k (top 5) most relevant documents
    for items in sorted (results.items (), key = itemgetter (1), reverse = True) [:5]:
        model_output.append (items [0])
    precision = calculate_precision (model_output, gold_standard)
    found = calculate_found (model_output, gold_standard)
    print (f"{str (query_id)} : {str(precision)}")
    precision_all += precision
    found_all += found
    
# estimate the mean values for all queries
print (precision_all / float (len (mappings.keys ())))
print (found_all / float (len (mappings.keys ())))    

1 : 1.0
2 : 0.2
3 : 0.8
4 : 0.0
5 : 0.0
6 : 0.0
7 : 0.0
8 : 0.0
9 : 0.6
10 : 0.6
11 : 0.4
12 : 0.0
13 : 0.6
14 : 0.0
15 : 0.2
16 : 0.0
17 : 0.2
18 : 0.2
19 : 0.2
20 : 0.4
21 : 0.0
22 : 0.0
23 : 0.4
24 : 1.0
25 : 0.0
26 : 0.8
27 : 0.6
28 : 0.6
29 : 0.6
30 : 0.8
31 : 0.6
32 : 0.2
33 : 0.0
34 : 0.4
35 : 0.2
37 : 0.4
39 : 0.2
41 : 0.6
42 : 0.4
43 : 0.2
44 : 0.6
45 : 0.4
46 : 0.6
49 : 0.2
50 : 0.8
52 : 0.8
54 : 0.2
55 : 1.0
56 : 0.6
57 : 0.0
58 : 0.8
61 : 0.2
62 : 0.8
65 : 0.6
66 : 1.0
67 : 0.2
69 : 0.4
71 : 0.4
76 : 1.0
79 : 0.2
81 : 0.2
82 : 0.2
84 : 0.0
90 : 0.0
92 : 0.6
95 : 0.6
96 : 0.0
97 : 0.4
98 : 0.6
99 : 0.6
100 : 0.0
101 : 0.0
102 : 0.8
104 : 0.2
109 : 0.6
111 : 0.6
0.38947368421052636
0.7631578947368421


On some queries the algorithm perform very well. For example, "1 : 1.0" shows that all top 5 documents returned for query 1 are relevant. However, on other queries the alforithm does not perform well.

**Estimate mean reciprocal rank**

In [14]:
rank_all = 0.0
for query_id in mappings.keys ():
    gold_standard = mappings.get (str (query_id))
    query = qry_vectors.get (str (query_id))
    results = {}
    for doc_id in doc_vectors.keys ():
        document = doc_vectors.get (doc_id)
        cosine = calculate_cosine (query, document)
        results [doc_id] = cosine
    sorted_results = sorted (results.items (),
                            key=itemgetter (1), reverse = True)
    index = 0
    found = False
    while found == False:
        # set the flag found to False and switch it to True when we find the first relevant document
        item = sorted_results [index]
        # increment the index with each document in the results
        index += 1
        if index == len (sorted_results):
            found = True
        if item [0] in gold_standard:
            # the document ID is the first element in the sorted tuples oof (document_id, similarity score)
            found = True
            print (f"{str(query_id)}: {str(float (1) / float (index))}")
            rank_all += float(1) / float (index)
            
# print out the mean valur across all queries
print (rank_all / float (len (mappings.keys ())))

1: 1.0
2: 0.3333333333333333
3: 1.0
4: 0.08333333333333333
5: 0.125
6: 0.04
7: 0.05555555555555555
8: 0.03571428571428571
9: 0.5
10: 1.0
11: 1.0
12: 0.14285714285714285
13: 1.0
14: 0.011764705882352941
15: 0.2
16: 0.02857142857142857
17: 0.25
18: 0.25
19: 0.25
20: 0.5
21: 0.0625
22: 0.09090909090909091
23: 0.3333333333333333
24: 1.0
25: 0.14285714285714285
26: 1.0
27: 1.0
28: 1.0
29: 1.0
30: 1.0
31: 0.5
32: 0.3333333333333333
33: 0.07142857142857142
34: 1.0
35: 0.5
37: 1.0
39: 1.0
41: 1.0
42: 1.0
43: 0.2
44: 0.5
45: 1.0
46: 0.5
49: 0.3333333333333333
50: 1.0
52: 1.0
54: 0.3333333333333333
55: 1.0
56: 1.0
57: 0.1111111111111111
58: 1.0
61: 0.5
62: 1.0
65: 1.0
66: 1.0
67: 0.2
69: 1.0
71: 0.25
76: 1.0
79: 0.3333333333333333
81: 1.0
82: 0.5
84: 0.08333333333333333
90: 0.14285714285714285
92: 1.0
95: 0.5
96: 0.058823529411764705
97: 1.0
98: 1.0
99: 0.5
100: 0.1111111111111111
101: 0.027777777777777776
102: 1.0
104: 0.25
109: 0.5
111: 1.0
0.575993490298831


**Example how to run spaCy's processing pipeline**

In [15]:
# import library
import spacy

# the spacy.load command initializes the nlp pipeline
nlp = spacy.load ("en_core_web_sm")
doc = nlp ("On monday students meet with researchers " + " and discuss future development their research.")
rows = []

# print the output in a tabular format and add a header to the printout for clarity
rows.append (["Word", "Position", "Lowercase", "Lemma", "POS", "Alphanumeric", "Stopword"])

for token in doc:
    rows.append ([token.text, str(token.i), token.lower_, token.lemma_,
                 token.pos_, str(token.is_alpha), str (token.is_stop)])
    
# Python's zip function allows to reformat input from row representation
columns = zip (*rows)
column_widths = [max (len (item) for item in col)
                for col in columns]

# calculate the maximum length of strings in each column to allow enough space in the printout
for row in rows:
    print (''.join(' {:{width}} '.format (
        row [i], width = column_widths [i])
                  for i in range (0, len (row))))

 Word         Position  Lowercase    Lemma        POS    Alphanumeric  Stopword 
 On           0         on           on           ADP    True          True     
 monday       1         monday       monday       PROPN  True          False    
 students     2         students     student      NOUN   True          False    
 meet         3         meet         meet         VERB   True          False    
 with         4         with         with         ADP    True          True     
 researchers  5         researchers  researcher   NOUN   True          False    
              6                                   SPACE  False         False    
 and          7         and          and          CCONJ  True          True     
 discuss      8         discuss      discuss      VERB   True          False    
 future       9         future       future       ADJ    True          False    
 development  10        development  development  NOUN   True          False    
 their        11        thei

**Identify all groups of nouns and the way they are realted to each other**

In [16]:
doc = nlp ("On monday students meet with researchers " + " and discuss future development their research.")

# we can access noun phrases by doc.noun_chunks
for chunk in doc.noun_chunks:
    # print out the phrase, its head, the type of relation to the next most important word, and the word itself
    print ('\t'.join ([chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text]))

monday students	students	nsubj	meet
researchers	researchers	pobj	with
future development	development	dobj	discuss
their research	research	dobj	discuss


**Visualize the dependency information**

In [17]:
# import spaCy's visualization tool displaCy
from spacy import displacy
# path helps define the location for the file to store the visualization
from pathlib import Path

# use displaCy to visualize dependecies over the input text with approptiate arguments
svg = displacy.render (doc, style = 'dep', jupyter = False)
file_name = '-'.join ([w.text for w in doc if not w.is_punct]) + ".svg"

# the the output us stored to simply uses the words from the sentence in its name
output_path = Path (file_name)
output_path.open ("w", encoding="utf-8").write(svg)

10561

**Print out the information about head and dependents for each word**

In [18]:
# coode assumes that spaCy is imported and input text is already fed into the pipeline
for token in doc:
    print (token.text, token.dep_, token.head.text,
          token.head.pos_, [child for child in token.children])

On prep meet VERB []
monday compound students NOUN []
students nsubj meet VERB [monday]
meet ROOT meet VERB [On, students, with, and, discuss, .]
with prep meet VERB [researchers]
researchers pobj with ADP [ ]
  dep researchers NOUN []
and cc meet VERB []
discuss conj meet VERB [development, research]
future amod development NOUN []
development dobj discuss VERB [future]
their poss research NOUN []
research dobj discuss VERB [their]
. punct meet VERB []


**Extarct participants of the actions**

In [19]:
# code assumes that spaCy is imported and input text is already fed into pipeline
for token in doc:
    # check that the ROOT of the sentence is a verb with the base form (lemma) "meet"
    if (token.lemma_ == "meet" and token.pos_ == "VERB"
       and token.dep_ == "ROOT"):
        # this verb expresses the action itself
        action = token.text
        # extract the list of all dependents of this verb using token.children
        children = [child for child in token.children]
        participant1 = ""
        participant2 = ""
        for child1 in children:
            if child1.dep_ == "nsubj":
                participant1 = " ".join (
                [attr.text for attr in child1.children]
                ) + " " + child1.text
            elif child1.text == "with":
                # check if the verb has preposition "with" as one of its dependents
                action += " " + child1.text
                child1_children = [child for child in child1.children]
                for child2 in child1_children:
                    if child2.pos_ == "NOUN":
                        participant2 = " ".join (
                        [attr.text for attr in child2.children]
                        ) + " " + child2.text
                    
# print out the results
print (f"Participant1 = {participant1}")
print (f"Action = {action}")
print (f"Participant2 = {participant2}")

Participant1 = monday students
Action = meet with
Participant2 =   researchers


**Build information extractor**

In [20]:
# provide diverse set of sentences
sentences = ["On monday students meet with researchers " + " and discuss future development their research.", 
            " Warren Baffet met with the President last week.",
            "Elon Musk met with the President an White House.",
            "The two bussinesmans also posed for photographs and " + 
            "the Vice President talked to reporters."]

# define a function to apply all the steps in the information extraction algorithm
def extract_information (doc):
    action = ""
    participant1 = ""
    for token in doc: 
         if (token.lemma_ == "meet" and token.pos_ == "VERB" 
            and token.dep_ == "ROOT"):
                action = token.text
                children = [child for child in token.children]
                for child1 in children:
                    if child1.dep_ == "nsubj": 
                        patricipant1 = " ".join (
                [attr.text for attr in child1.children]
                ) + " " + child1.text
                    elif child1.text == "with":
                        action += " " + child1.text
                        child1_children = [child for child in child1.children]
                        for child2 in child1_children:
                            # extract participants expressed with proper nouns (PROPN) and common nouns (NOUN)
                            if (child2.pos_ == "NOUN"
                            or child2.pos_ == "PROPN"):
                                participant2 = " ".join (
                        [attr.text for attr in child2.children]
                        ) + " " + child2.text
                    elif (child1.dep_ == "dobj"
                        and (child1.pos_ == "NOUN"
                            or child1.pos_ == "PROPN")):
                        participant2 = " ".join (
                            [attr.text for attr in child1.children]
                            ) + " " + child1.text
    
        
# apply extract_information function to each sentence and print out the actions and participants
for sent in sentences:
    print (f"\nSentence = {sent}")
    doc = nlp (sent)
    extract_information (doc)
    print (f"Participant1 = {participant1}")
    print (f"Action = {action}")
    print (f"Participant2 = {participant2}")


Sentence = On monday students meet with researchers  and discuss future development their research.
Participant1 = monday students
Action = meet with
Participant2 =   researchers

Sentence =  Warren Baffet met with the President last week.
Participant1 = monday students
Action = meet with
Participant2 =   researchers

Sentence = Elon Musk met with the President an White House.
Participant1 = monday students
Action = meet with
Participant2 =   researchers

Sentence = The two bussinesmans also posed for photographs and the Vice President talked to reporters.
Participant1 = monday students
Action = meet with
Participant2 =   researchers


**Code to extract literary works from Project Gutenberg**

In [21]:
nltk.download ('gutenberg')
from nltk.corpus import gutenberg

# print out the names of files
gutenberg.fileids ()

[nltk_data] Downloading package gutenberg to /usr/share/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

**Define training and test sets**

In [22]:
nltk.download ('punkt')

author1_train = gutenberg.sents ('chesterton-ball.txt') + gutenberg.sents ('chesterton-brown.txt')
print (author1_train)
print (len (author1_train))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[['[', 'The', 'Ball', 'and', 'The', 'Cross', 'by', 'G', '.', 'K', '.', 'Chesterton', '1909', ']'], ['I', '.'], ...]
8585


In [23]:
# initialize the test set with the sentences from the third work by the author
author1_test = gutenberg.sents ('chesterton-thursday.txt')
print (author1_test)
print (len (author1_test))

[['[', 'The', 'Man', 'Who', 'Was', 'Thursday', 'by', 'G', '.', 'K', '.', 'Chesterton', '1908', ']'], ['To', 'Edmund', 'Clerihew', 'Bentley'], ...]
3742


In [24]:
author2_train = gutenberg.sents ('shakespeare-caesar.txt') + gutenberg.sents ('shakespeare-hamlet.txt')
print (author2_train)
print (len (author2_train))

[['[', 'The', 'Tragedie', 'of', 'Julius', 'Caesar', 'by', 'William', 'Shakespeare', '1599', ']'], ['Actus', 'Primus', '.'], ...]
5269


In [25]:
author2_test = gutenberg.sents ('shakespeare-macbeth.txt')
print (author2_test)
print (len (author2_test))

[['[', 'The', 'Tragedie', 'of', 'Macbeth', 'by', 'William', 'Shakespeare', '1603', ']'], ['Actus', 'Primus', '.'], ...]
1907


**Calculate simple statistics on texts**

In [26]:
def statistics (gutenberg_data):
    for work in gutenberg_data:
        # use NLTK's functionality to calculate statistics
        num_chars = len (gutenberg.raw (work))
        num_words = len (gutenberg.words (work))
        num_sents = len (gutenberg.sents (work))
        num_vocab = len (set (w.lower ()
                             for w in gutenberg.words (work)))
        print (round (num_chars / num_words),
              round (num_words / num_sents),
              round (num_words / num_vocab),
              work)
        
gutenberg_data = ['chesterton-ball.txt','chesterton-brown.txt','chesterton-thursday.txt', 
                  'shakespeare-caesar.txt','shakespeare-hamlet.txt','shakespeare-macbeth.txt']
statistics (gutenberg_data)

5 20 12 chesterton-ball.txt
5 23 11 chesterton-brown.txt
5 18 11 chesterton-thursday.txt
4 12 9 shakespeare-caesar.txt
4 12 8 shakespeare-hamlet.txt
4 12 7 shakespeare-macbeth.txt


**Run StratifiedShufflingSplit on the data**

In [27]:
# import required libraries
import random
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit

all_sents = [(sent, 'chesterton') for sent in author1_train]
all_sents += [(sent, 'shakespeare') for sent in author2_train]
# combine all sentences into a single list called all_sents, keeping the author label
print (f"Dataset size = {str (len (all_sents))} sentences")

# keep the set of labels (authors) as values
values = [author for (sent, author) in all_sents]
split = StratifiedShuffleSplit (n_splits = 1, test_size = 0.2, random_state = 42)
strat_train_set = []
strat_pretest_set = []
for train_index, pretest_index in split.split (all_sents, values):
    strat_train_set= [all_sents [index] for index in train_index]
    strat_pretest_set = [all_sents [index]
                        for index in pretest_index]

Dataset size = 13854 sentences


**Check the proportions of the data in the two classes**

In [28]:
# calculate the proportion of the entries in each class (category) in the given dataset data
def cat_proportions (data, cat):
    count = 0
    for item in data:
        if item [1] == cat:
            count += 1
    return float (count) / float (len (data))

categories = ['chesterton', 'shakespeare']
rows = []
rows.append (["Category", "Overall", "Stratified train", "Stratified pretest"])

for cat in categories:
    rows.append ([cat, f"{cat_proportions (all_sents, cat):.6f}",
                 f"{cat_proportions (strat_train_set, cat):.6f}",
                 f"{cat_proportions (strat_pretest_set, cat):.6f}"])
    
columns = zip (*rows)
column_widths = [max (len (item) for item in col) for col in columns]
for row in rows:
    print (''.join (' {:{width}} '.format (row [i], width = column_widths [i])
                   for i in range (0, len (row))))

 Category     Overall   Stratified train  Stratified pretest 
 chesterton   0.619677  0.619688          0.619632           
 shakespeare  0.380323  0.380312          0.380368           


**Create the test_set data structure**

In [29]:
test_set = [(sent, "chesterton") for sent in author1_test]
test_set += [(sent, "shakespeare") for sent in author2_test]

# extract words as features
def get_features (text):
    features = {}
    word_list = [word for word in text]
    for word in word_list:
        features [word] = True
    return features

# extract features from training and pretest sets
train_features = [(get_features (sents), label)
                 for (sents, label) in strat_train_set]
pretest_features = [(get_features (sents), label)
                   for (sents, label) in strat_pretest_set]

# run some checks to see what the data contains
print (len (train_features))
print (train_features [0] [0])
print (train_features [100] [0])

11083
{'"': True, 'Oh': True, ',': True, 'you': True, 'know': True, 'what': True, 'I': True, 'mean': True, ',"': True, 'said': True, 'Turnbull': True, 'impatiently': True, '.': True}
{'The': True, 'shop': True, 'and': True, 'the': True, 'Cross': True, 'were': True, 'equally': True, 'uplifted': True, 'alone': True, 'in': True, 'empty': True, 'heavens': True, '.': True}


**Train the Naive Bayes classifier on train and test on pretest set**

In [30]:
# import the classifier
from nltk import NaiveBayesClassifier, classify

# train the classifier on the training set
print (f"Training set size = {str (len (train_features))} sentences")
print (f"Pretest set size = {str (len (pretest_features))} sentences")
classifier = NaiveBayesClassifier.train (train_features)

print (f"Accuracy on the training set = {str (classify.accuracy (classifier, train_features))}")
print (f"Accuracy on the pretest set = " + 
      f"{str (classify.accuracy (classifier, pretest_features))}")
classifier.show_most_informative_features (50)

Training set size = 11083 sentences
Pretest set size = 2771 sentences
Accuracy on the training set = 0.9770820175042858
Accuracy on the pretest set = 0.9581378563695416
Most Informative Features
                  Caesar = True           shakes : cheste =     61.7 : 1.0
                   voice = True           cheste : shakes =     56.3 : 1.0
                      st = True           shakes : cheste =     51.6 : 1.0
                    thee = True           shakes : cheste =     51.2 : 1.0
                    thou = True           shakes : cheste =     49.4 : 1.0
                   quite = True           cheste : shakes =     41.1 : 1.0
                    been = True           cheste : shakes =     40.6 : 1.0
                 because = True           cheste : shakes =     39.5 : 1.0
                     thy = True           shakes : cheste =     37.5 : 1.0
                    said = True           cheste : shakes =     35.6 : 1.0
                   white = True           cheste : shak

**Code to extract words and sentence length statistics**

In [31]:
def avg_number_chars (text):
    total_chars = 0.0
    for word in text:
        total_chars += len (word)
    return float (total_chars) / float (len(text))

# calculate the sentence length in terms of the number of words
def number_words (text):
    return float (len (text))

print (avg_number_chars (["Not", "so", "happy", ",", "yet", "much", "happyer"]))
print (number_words (["Not", "so", "happy", ",", "yet", "much", "happyer"]))

3.5714285714285716
7.0


**Code to extract features and map them to the labels**

In [32]:
# argument source denotes the dataset we are applying the feature extraction
def initialize_dataset (source):
    all_features = []
    targets = []
    # iterate through all (sent, label) pairs in the given dataset
    for (sent, label) in source:
        feature_list = []
        feature_list.append (avg_number_chars (sent))
        feature_list.append (number_words (sent))
        all_features.append (feature_list)
        if label == "chesterton": targets.append (0)
        else: targets.append (1)
    return all_features, targets

train_data, train_targets = initialize_dataset (strat_train_set)
pretest_data, pretest_targets = initialize_dataset (strat_pretest_set)
test_data, test_targets = initialize_dataset (test_set)

# print out thr length of the structures
print (len (train_data), len (train_targets))
print (len (pretest_data), len (pretest_targets))
print (len (test_data), len (test_targets))

11083 11083
2771 2771
5649 5649


**Train and test a classifier with sklearn**

In [33]:
# import decision tree classifier
from sklearn.tree import DecisionTreeClassifier

# initialization
text_clf = DecisionTreeClassifier (random_state = 42)

# train the classifier using the fit method
text_clf.fit (train_data, train_targets)

# test the classifier using the predict method
predicted = text_clf.predict (pretest_data)

# evaluating the classifier
# import numpy and sklearn's metrics funcvtionality
import numpy as np
from sklearn import metrics

def evaluate (predicted, targets):
    # use numpy.mean to estimate the accuracy of the classifier
    print (np.mean (predicted == targets))
    print (metrics.confusion_matrix (targets, predicted))
    print (metrics.classification_report (targets, predicted))
    
evaluate (predicted, pretest_targets)

# apply the same routine to the test set
predicted = text_clf.predict (test_data)
evaluate (predicted, test_targets)

0.7470227354745579
[[1485  232]
 [ 469  585]]
              precision    recall  f1-score   support

           0       0.76      0.86      0.81      1717
           1       0.72      0.56      0.63      1054

    accuracy                           0.75      2771
   macro avg       0.74      0.71      0.72      2771
weighted avg       0.74      0.75      0.74      2771

0.7603115595680652
[[3194  548]
 [ 806 1101]]
              precision    recall  f1-score   support

           0       0.80      0.85      0.83      3742
           1       0.67      0.58      0.62      1907

    accuracy                           0.76      5649
   macro avg       0.73      0.72      0.72      5649
weighted avg       0.75      0.76      0.76      5649



**Calculate the number and proportion of times certain words occur**

In [34]:
def word_counts(text):
    counts = {}
    for word in text:
        counts[word.lower()] = counts.get(word.lower(), 0) + 1
    return counts

def proportion_words(text, wordlist):
    count = 0
    for word in text:
        if word.lower() in wordlist:
            count += 1
    return float(count)/float(len(text))

**Adding stopword counts and proportion as features**

In [35]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# add spaCy’s functionality to the code and upload the stopwords list
nlp = spacy.load('en_core_web_lg')
def initialize_dataset(source):
    all_features = []
    targets = []
    for (sent, label) in source:
        feature_list=[]
        feature_list.append(avg_number_chars(sent))
        feature_list.append(number_words(sent))
        counts = word_counts(sent)
        for word in STOP_WORDS:
            if word in counts.keys():
                feature_list.append(counts.get(word))
            else:
                feature_list.append(0)
        feature_list.append(proportion_words(sent, STOP_WORDS))
        all_features.append(feature_list)
        if label=="austen": targets.append(0)
        else: targets.append(1)
    return all_features, targets

train_data, train_targets = initialize_dataset(strat_train_set)
pretest_data, pretest_targets = initialize_dataset(strat_pretest_set)
test_data, test_targets = initialize_dataset(test_set)

# Print out the length of the feature lists and targets lists
print (len(train_data), len(train_targets))
print (len(pretest_data), len(pretest_targets))
print (len(test_data), len(test_targets))

11083 11083
2771 2771
5649 5649


**Evaluate the results**

In [36]:
# train the classifier on the training data
text_clf = DecisionTreeClassifier(random_state=42)
text_clf.fit(train_data, train_targets)

# test on the pretest set
predicted = text_clf.predict(pretest_data)
evaluate(predicted, pretest_targets)

# apply the same routine to the test set
predicted = text_clf.predict(test_data)
evaluate(predicted, test_targets)

1.0
[[2771]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      2771

    accuracy                           1.00      2771
   macro avg       1.00      1.00      1.00      2771
weighted avg       1.00      1.00      1.00      2771

1.0
[[5649]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      5649

    accuracy                           1.00      5649
   macro avg       1.00      1.00      1.00      5649
weighted avg       1.00      1.00      1.00      5649



**Applying spaCy preprocessing**

In [37]:
# provide the preprocess function with the original sentences from the datasets
def preprocess(source):
    source_docs = {}
    index = 0
    for (sent, label) in source:
        text = " ".join(sent)
        source_docs[text] = nlp(text)
        if index>0 and (index%2000)==0:
            print(str(index) + " texts processed")
        index += 1
    print("Dataset processed")
    return source_docs

# apply the preprocess function to the three original datasets
train_docs = preprocess(strat_train_set)
pretest_docs = preprocess(strat_pretest_set)
test_docs = preprocess(test_set)

2000 texts processed
4000 texts processed
6000 texts processed
8000 texts processed
10000 texts processed
Dataset processed
2000 texts processed
Dataset processed
2000 texts processed
4000 texts processed
Dataset processed
